In [2]:
require 'csv'
require 'distribution'

true

In [3]:
class SeedStock
  attr_accessor :seed, :gene_id, :last_planted, :storage, :grams_remaining

  def initialize(seed, gene_id, last_planted, storage, grams_remaining)
    @seed = seed
    @gene_id = gene_id
    @last_planted = last_planted
    @storage = storage
    @grams_remaining = grams_remaining.to_i
    
  end

  def planting_simulation(amount)
    @grams_remaining -= amount
    @grams_remaining = [0, @grams_remaining].max
    puts "WARNING: we have run out of Seed Stock #{@seed}" if @grams_remaining == 0
  end

  def to_csv
    [@seed, @gene_id, @last_planted, @storage, @grams_remaining]
  end

end

:to_csv

In [13]:
class Gene
  attr_accessor :gene_id, :gene_name, :mutant_phenotype

  def initialize(gene_id, gene_name)
    @gene_id = gene_id
    @gene_name = gene_name 
    @mutant_phenotype = mutant_phenotype

  end

  def self.build_gene_id_to_gene_name_mapping
    gene_info_data = CSV.read('StockDatabaseDataFiles/gene_information.tsv', col_sep: "\t", headers: true)
    gene_id_to_gene_name = {}
    gene_info_data.each do |row|
      gene_id = row['Gene_ID']
      gene_name = row['Gene_name']
      gene_id_to_gene_name[gene_id] = gene_name
    end
    gene_id_to_gene_name
  end
end

:build_gene_id_to_gene_name_mapping

In [15]:
class HybridCross
  attr_accessor :parent1, :parent2, :f2_wild, :f2_p1, :f2_p2, :f2_p1p2
  attr_accessor :seed_stock_parent1, :seed_stock_parent2
  attr_accessor :chi_squared, :statistically_significant, :gene_name_parent1, :gene_name_parent2

  def initialize(parent1, parent2, f2_wild, f2_p1, f2_p2, f2_p1p2)
    @parent1 = parent1
    @parent2 = parent2
    @f2_wild = f2_wild.to_i
    @f2_p1 = f2_p1.to_i
    @f2_p2 = f2_p2.to_i
    @f2_p1p2 = f2_p1p2.to_i
    @seed_stock_parent1 = nil
    @seed_stock_parent2 = nil
    @chi_squared = 0.0
    @statistically_significant = false
    @gene_name_parent1 = nil
    @gene_name_parent2 = nil
  end

 def link_to_seed_stocks_and_genes_names(seed_stocks, gene_id_to_gene_name)
    @seed_stock_parent1 = seed_stocks.find { |seed_stock| seed_stock.seed == parent1 }
    @seed_stock_parent2 = seed_stocks.find { |seed_stock| seed_stock.seed == parent2 }
    
    @gene_name_parent1 = gene_id_to_gene_name[@seed_stock_parent1.gene_id]
    @gene_name_parent2 = gene_id_to_gene_name[@seed_stock_parent2.gene_id]
  end

  def chi_squared_test
    observed_counts = [f2_wild, f2_p1, f2_p2, f2_p1p2]
    total_observations = observed_counts.sum

    expected_counts = [
      total_observations * 9 / 16.0,
      total_observations * 3 / 16.0,
      total_observations * 3 / 16.0,
      total_observations * 1 / 16.0
    ]

    @chi_squared = observed_counts.zip(expected_counts).sum do |observed, expected|
      ((observed - expected)**2) / expected
    end

    degrees_of_freedom = observed_counts.size - 1
    p_value = 1 - Distribution::ChiSquare.cdf(@chi_squared, degrees_of_freedom)

    @statistically_significant = p_value < 0.05
    @chi_squared
  end
end

:chi_squared_test

In [9]:


# Create an array to hold instances of SeedStock
seed_stocks = []

# Read the data from the input file
CSV.foreach('StockDatabaseDataFiles/seed_stock_data.tsv', col_sep: "\t", headers: true) do |row|
  # Create a SeedStock object for each row and store it in the array
  seed_stock = SeedStock.new(row['Seed_Stock'], row['Mutant_Gene_ID'], row['Last_Planted'], row['Storage'], row['Grams_Remaining'])
  seed_stocks << seed_stock
end

# Simulate planting 7 grams of seeds and update the gene bank information
seed_stocks.each { |seed_stock| seed_stock.planting_simulation(7) }

# Specify the path to the output TSV file
output_file = 'new_stock_file.tsv'

# Write the updated data to the output file
CSV.open(output_file, 'w', col_sep: "\t") do |csv|
  # Write the header row
  csv << ['Seed_Stock', 'Mutant_Gene_ID', 'Last_Planted', 'Storage', 'Grams_Remaining']
  # Write the data for each SeedStock object to the file
  seed_stocks.each { |seed_stock| csv << seed_stock.to_csv }
end

puts "Data saved to #{output_file}"

Data saved to new_stock_file.tsv


In [19]:
# Read data from seed_stock.tsv and link with gene names
seed_stock_data = CSV.read('StockDatabaseDataFiles/seed_stock_data.tsv', col_sep: "\t", headers: true)
seed_stocks = seed_stock_data.map do |row|
  gene_id = row['Mutant_Gene_ID']
  seed_stock = SeedStock.new(row['Seed_Stock'], gene_id, row['Last_Planted'], row['Storage'], row['Grams_Remaining'])
  seed_stock
end

# Read data from cross_data.tsv
cross_data = CSV.read('StockDatabaseDataFiles/cross_data.tsv', col_sep: "\t", headers: true)

# Create an array to store instances of the HybridCross class
hybrid_crosses = []

gene_id_to_gene_name = Gene.build_gene_id_to_gene_name_mapping

# Iterate through the CSV data and create instances of HybridCross
cross_data.each do |row|
  hybrid_cross = HybridCross.new(
    row['Parent1'],
    row['Parent2'],
    row['F2_Wild'],
    row['F2_P1'],
    row['F2_P2'],
    row['F2_P1P2']
  )
  hybrid_cross.link_to_seed_stocks_and_genes_names(seed_stocks, gene_id_to_gene_name)
  chi_squared = hybrid_cross.chi_squared_test
  hybrid_cross.chi_squared = chi_squared
  hybrid_crosses << hybrid_cross
end

# Link gene names to hybrid crosses
statistically_significant_genes = hybrid_crosses.select { |gene| gene.statistically_significant }

puts "Recording:"
statistically_significant_genes.each do |gene|
  parent1_gene_name = gene.gene_name_parent1
  parent2_gene_name = gene.gene_name_parent2
  chi_squared = gene.chi_squared
  puts "#{parent1_gene_name} is linked to #{parent2_gene_name} with chi-squared score #{format('%.7f', chi_squared)}"

puts
# Print the names of statistically significant genes
puts "Final Report:"

  puts "#{parent1_gene_name} is linked to #{parent2_gene_name}"
  

  # Add the reverse direction as well
  puts "#{parent2_gene_name} is linked to #{parent1_gene_name}"
  
end

Recording:
ufo is linked to pi with chi-squared score 32.2794279

Final Report:
ufo is linked to pi
pi is linked to ufo


[#<#<Class:0x00005606e1e533d0>::HybridCross:0x00005606e18eac40 @parent1="A51", @parent2="B52", @f2_wild=152, @f2_p1=26, @f2_p2=22, @f2_p1p2=2, @seed_stock_parent1=#<#<Class:0x00005606e1e533d0>::SeedStock:0x00005606e1703580 @seed="A51", @gene_id="AT1G30950", @last_planted="9/2/2013", @storage="cama25", @grams_remaining=5>, @seed_stock_parent2=#<#<Class:0x00005606e1e533d0>::SeedStock:0x00005606e1703080 @seed="B52", @gene_id="AT5G20240", @last_planted="8/11/2012", @storage="cama16", @grams_remaining=7>, @chi_squared=32.27942794279428, @statistically_significant=true, @gene_name_parent1="ufo", @gene_name_parent2="pi">]